In [1]:
import  random

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
sc = SparkContext(appName="YourTest", master="local[2]", conf=SparkConf().set('spark.ui.port', random.randrange(4000,5000)))
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [3]:
file_location = "DataTable.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

In [4]:
df = df.withColumn("DKSalary",df["DKSalary"].cast(IntegerType()))

df = df.filter((df.DKPos != "#N/A") & (df.FDPos != "#N/A") & (df.DKFP != "#N/A") & (df.FDFP != "#N/A"))
df = df.withColumn("DKFP",df["DKFP"].cast(IntegerType()))

In [5]:
pos_dict={"C":[],"W":[],"D":[],"G":[]}
for item in  df.collect():
    if item["DKPos"] in pos_dict:
        pos_dict[item["DKPos"]].append({"DKSalary":item["DKSalary"],"FDFP":item["FDFP"],"PlayerID":item["PlayerID"]})

In [12]:
for pos,item in  pos_dict.items():
    pos_dict[pos]=sorted(item, key=lambda x: (float(x["FDFP"]),-float(x["DKSalary"])), reverse=True) 

In [8]:
pos_dict["G"]

[{'DKSalary': 8000, 'FDFP': '59.2', 'PlayerID': '5452'},
 {'DKSalary': 7800, 'FDFP': '57.6', 'PlayerID': '5420'},
 {'DKSalary': 7100, 'FDFP': '56.8', 'PlayerID': '3904177'},
 {'DKSalary': 7000, 'FDFP': '56', 'PlayerID': '3069266'},
 {'DKSalary': 7600, 'FDFP': '55.2', 'PlayerID': '5734'},
 {'DKSalary': 7400, 'FDFP': '54.4', 'PlayerID': '5452'},
 {'DKSalary': 7900, 'FDFP': '54.4', 'PlayerID': '4063288'},
 {'DKSalary': 7600, 'FDFP': '52.8', 'PlayerID': '3020225'},
 {'DKSalary': 7800, 'FDFP': '52.8', 'PlayerID': '3069285'},
 {'DKSalary': 8200, 'FDFP': '52.8', 'PlayerID': '3870'},
 {'DKSalary': 8200, 'FDFP': '52.8', 'PlayerID': '3151038'},
 {'DKSalary': 8300, 'FDFP': '52.8', 'PlayerID': '2976847'},
 {'DKSalary': 8200, 'FDFP': '52', 'PlayerID': '2590874'},
 {'DKSalary': 7500, 'FDFP': '51.2', 'PlayerID': '5096'},
 {'DKSalary': 7000, 'FDFP': '50.4', 'PlayerID': '4239677'},
 {'DKSalary': 7800, 'FDFP': '50.4', 'PlayerID': '5111'},
 {'DKSalary': 7300, 'FDFP': '49.6', 'PlayerID': '3042109'},
 {'DK

In [13]:
budget=50000
num_dict={"G":2,"C":3,"W":3,"D":2}
i=0
def get_player(i):
    while True:
        cost=0
        result_dict={"G":[],"C":[],"W":[],"D":[]}
        for pos in ["G","C","W","D"]:
            item=pos_dict[pos]
            if pos=="G":
                item=item[i:]
            if pos=="C":
                item=item[i:]
            for value in item:
                if len(result_dict[pos])<num_dict[pos] and cost<=budget:
                    result_dict[pos].append(value)
                    cost+=value["DKSalary"]
                if len(result_dict["D"])==num_dict["D"] and len(result_dict["G"])==num_dict["G"] \
                and len(result_dict["W"])==num_dict["W"] and len(result_dict["C"])==num_dict["C"] and cost<=budget :return result_dict,cost

        if len(result_dict["D"])==num_dict["D"] and len(result_dict["G"])==num_dict["G"] \
                and len(result_dict["W"])==num_dict["W"] and len(result_dict["C"])==num_dict["C"] and cost<=budget:return result_dict,cost
        i+=1
#         print(i)

result_dict,cost=get_player(i)
print(result_dict)
print(cost)

{'G': [{'DKSalary': 7200, 'FDFP': '42.4', 'PlayerID': '2339'}, {'DKSalary': 7300, 'FDFP': '42.4', 'PlayerID': '3042109'}], 'C': [{'DKSalary': 2500, 'FDFP': '40.4', 'PlayerID': '3025616'}, {'DKSalary': 3800, 'FDFP': '40', 'PlayerID': '3899933'}, {'DKSalary': 4600, 'FDFP': '40', 'PlayerID': '5798'}], 'W': [{'DKSalary': 3800, 'FDFP': '64.5', 'PlayerID': '3900147'}, {'DKSalary': 4500, 'FDFP': '63.4', 'PlayerID': '3227'}, {'DKSalary': 4900, 'FDFP': '62.1', 'PlayerID': '3249'}], 'D': [{'DKSalary': 4500, 'FDFP': '64.8', 'PlayerID': '3114769'}, {'DKSalary': 5400, 'FDFP': '50.4', 'PlayerID': '5079'}]}
48500
